Задание 1.
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)
С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?
Повторим шаги из заданий 1 и 2, используя библиотеку nltk.


Задание 2.
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

Задание 3.
Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.


In [1]:
from tqdm import tqdm
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import pandas as pd
import numpy as np

In [2]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.0/en_core_web_md-2.2.0.tar.gz
import spacy
from spacy import displacy
import en_core_web_md

     |████████████████████████████████| 96.4MB 66kB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.0-cp36-none-any.whl size=98072934 sha256=46a563f549f124ed62e2474783ea6effa5cb9c8e0a805dbcaacac38a7fd9a3f6
  Stored in directory: /root/.cache/pip/wheels/5f/3e/c9/36dd6e13b449fd84cd1f94b72dfbc559daf09f53dbf4e697a3
Successfully built en-core-web-md


In [3]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('maxent_ne_chunker') 
nltk.download('words')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
tweets = pd.read_pickle('/content/drive/My Drive/NLP/HW3-new/combine_df_prepocessed.pkl')
tweets.head()

,id,label,tweet,clean_tweet,tweet_token,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,#model i love u take with u all the time in ...,model love yoyou take with yoyou all the time ...,"[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]","[model, love, yoyou, take, yoyou, time, yoyour]"
4,5,0.0,factsguide: society now #motivation,factsguide society now motivation,"[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


In [6]:
nlp = en_core_web_md.load()
nlp.max_length = 1000000

In [7]:
def ner_by_spacy(texts):

    entities = []
    labels = []
    for text in tqdm(texts):
        doc = nlp(text)
        for ent in doc.ents:
            entities.append(ent.text)
            labels.append(ent.label_)

    df = pd.DataFrame({'Entities': entities, 'Labels': labels})

    return df

In [8]:
def ner_by_nltk(texts):
    
    entities = []
    labels = []
    
    for text in tqdm(texts):
        words = nltk.word_tokenize(text)
        pos_tags = nltk.pos_tag(words)
        chunks = nltk.ne_chunk(pos_tags, binary=False)

        for chunk in chunks:
            if hasattr(chunk, 'label'):
                entities.append(' '.join(c[0] for c in chunk))
                labels.append(chunk.label())
    
    df = pd.DataFrame({'Entities': entities, 'Labels': labels})

    return df

In [9]:
def add_to_dict(dict_, word):
  if word in dict_.keys():
    dict_[word] += 1
  else:
    dict_[word] = 1

In [10]:
df_spacy = ner_by_spacy(tweets['clean_tweet'])

100%|██████████| 49159/49159 [08:16<00:00, 98.96it/s] 


In [11]:
df_gb = df_spacy.groupby(['Entities', 'Labels'])[['Entities']].count()

In [12]:
df_gb.columns = ['counts']
df_gb.reset_index(inplace=True)
df_gb.sort_values(by='counts', ascending=False).head(20)

,Entities,Labels,counts
20725,today,DATE,1596
22928,yoyou,PERSON,1161
20798,tomorrow,DATE,616
7038,friday,DATE,604
6675,first,ORDINAL,546
14917,one,CARDINAL,529
22966,yoyour,PERSON,505
18983,summer,DATE,503
19072,sunday,DATE,468
20815,tonight,TIME,436


In [13]:
df_gb_lb = df_spacy.groupby(['Labels'])[['Labels']].count()
df_gb_lb.columns = ['counts']
df_gb_lb.reset_index(inplace=True)
df_gb_lb.sort_values(by='counts', ascending=False).head()

,Labels,counts
13,PERSON,19827
1,DATE,11011
11,ORG,8181
4,GPE,5876
16,TIME,2282


In [14]:
border = 31963
train_df = tweets.loc[tweets.id < border]
test_df = tweets.loc[tweets.id > border]

In [15]:
train =  " ".join(train_df['clean_tweet'].values)
test =  " ".join(test_df['clean_tweet'].values)
train = train[:1000000]
test = test[:1000000]

In [16]:
train_person_dict = {}
train_org_dict = {}
test_person_dict = {}
test_org_dict = {}

In [17]:
doc = nlp(train)

In [18]:
for entity in tqdm(doc.ents):
  if entity.label_ == 'PERSON':
    add_to_dict(train_person_dict, entity.text)
  if entity.label_ == 'ORG':
    add_to_dict(train_org_dict, entity.text)
doc2 = nlp(test)

100%|██████████| 14255/14255 [00:00<00:00, 246059.71it/s]


In [19]:
for entity in tqdm(doc2.ents):
  if entity.label_ == 'PERSON':
    add_to_dict(test_person_dict, entity.text)
  if entity.label_ == 'ORG':
    add_to_dict(test_org_dict, entity.text)

100%|██████████| 14213/14213 [00:00<00:00, 266063.44it/s]


In [20]:
train_person_df = pd.DataFrame.from_dict(train_person_dict, columns=['cnt'], orient='index')
train_org_df = pd.DataFrame.from_dict(train_org_dict, columns=['cnt'], orient='index')
train_person_df.sort_values(by='cnt', ascending=False, inplace=True)
train_org_df.sort_values(by='cnt', ascending=False, inplace=True)

test_person_df = pd.DataFrame.from_dict(test_person_dict, columns=['cnt'], orient='index')
test_org_df = pd.DataFrame.from_dict(test_org_dict, columns=['cnt'], orient='index')
test_person_df.sort_values(by='cnt', ascending=False, inplace=True)
test_org_df.sort_values(by='cnt', ascending=False, inplace=True)

In [21]:
train_person_df.head(20)

,cnt
yoyou,360
bihday,99
yoyour,69
obama,34
bing bong,18
hillary,17
hu,17
allahsoil,17
bing bong bing bong,13
heabroken,13


In [22]:
test_person_df.head(20)

,cnt
yoyou,340
bihday,111
yoyour,59
obama,30
hu,19
allahsoil,18
fathersday,18
photooftheday,15
bing bong,14
bing bong bing bong,12


In [23]:
train_org_df.head(20)

,cnt
gop,18
nba,17
yoyoyou,16
netflix,14
islam,12
orlando,11
bihday,11
disney,11
fed,8
nyc,8


In [24]:
test_org_df.head(20)

,cnt
disney,18
gop,17
nba,14
aicle,13
isis,12
amazon,11
malevote,11
bihday,10
orlando,9
netflix,8


результат есть, но сказать что отличный нелзя

In [25]:
text =  " ".join(tweets['clean_tweet'].values)
text = text[:1000000]

In [26]:
# text = text.title()
# text = text.lower()
text = text.upper()

In [27]:
ent_nltk_dict = {}
ent_nltk_types_dict = {}

In [28]:
for chunk in tqdm(nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text)))):
  if hasattr(chunk, 'label'):
    add_to_dict(ent_nltk_dict, (' '.join(c[0] for c in chunk)))
    add_to_dict(ent_nltk_types_dict, chunk.label())

100%|██████████| 168451/168451 [00:00<00:00, 1208214.26it/s]


In [29]:
ent_nltk_types_df = pd.DataFrame.from_dict(ent_nltk_types_dict, columns=['cnt'], orient='index')
ent_nltk_df = pd.DataFrame.from_dict(ent_nltk_dict, columns=['cnt'], orient='index')
ent_nltk_types_df.sort_values(by='cnt', ascending=False, inplace=True)
ent_nltk_df.sort_values(by='cnt', ascending=False, inplace=True)

In [30]:
ent_nltk_df.head(20)

,cnt
THE,1332
FAMILY,199
TIME,184
HAPPINESS,172
GOLD,145
NOT,140
YOU,121
FATHERS,116
WELL,108
TO,106


In [31]:
ent_nltk_types_df.head(5)

,cnt
ORGANIZATION,18597
PERSON,76
GPE,14
GSP,1


In [32]:
train_nltk = train.title()
test_nltk = test.title()

In [33]:
train_person_dict = {}
train_org_dict = {}

In [34]:
for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(train_nltk))):
  if hasattr(chunk, 'label'):
    if chunk.label() == 'PERSON':
      add_to_dict(train_person_dict, (' '.join(c[0] for c in chunk)))
    if chunk.label() == 'ORGANIZATION':
      add_to_dict(train_org_dict, (' '.join(c[0] for c in chunk)))

In [35]:
train_person_df = pd.DataFrame.from_dict(train_person_dict, columns=['cnt'], orient='index')
train_org_df = pd.DataFrame.from_dict(train_org_dict, columns=['cnt'], orient='index')
train_person_df.sort_values(by='cnt', ascending=False, inplace=True)
train_org_df.sort_values(by='cnt', ascending=False, inplace=True)

In [36]:
test_person_dict = {}
test_org_dict = {}

for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(test_nltk))):
  if hasattr(chunk, 'label'):
    if chunk.label() == 'PERSON':
      add_to_dict(test_person_dict, (' '.join(c[0] for c in chunk)))
    if chunk.label() == 'ORGANIZATION':
      add_to_dict(test_org_dict, (' '.join(c[0] for c in chunk)))

In [37]:
test_person_df = pd.DataFrame.from_dict(test_person_dict, columns=['cnt'], orient='index')
test_org_df = pd.DataFrame.from_dict(test_org_dict, columns=['cnt'], orient='index')
test_person_df.sort_values(by='cnt', ascending=False, inplace=True)
test_org_df.sort_values(by='cnt', ascending=False, inplace=True)

In [38]:
train_person_df.head(20)

,cnt
Yoyou,139
Will,101
Delete Old,43
Happy,35
Forward,35
Libtard,33
Father,31
Grateful,27
Been,27
Sad,25


In [39]:
test_person_df.head(20)

,cnt
Yoyou,132
Will,97
Forward,40
Happy,40
Delete Old,34
Father,32
Ready,26
Libtard,24
Grateful,23
Good,22


In [40]:
train_org_df.head(20)

,cnt
Vicinity Of Their,25
Fathers,21
Lot,20
Vast Expanse Of Mountains,18
Well Deletetweets,18
House,16
All,15
Work Conference,14
Same,12
No,11


In [41]:
test_org_df.head(20)

,cnt
Vicinity Of Their,23
Vast Expanse Of Mountains,22
Lot,21
All,20
Happiness,20
Best,18
Work Conference,15
Same,15
House,15
Only,13


nltk явно хуже, и настраивается сложнее и количество распознаных существенно меньше